In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)
from conf import *

In [2]:
importlib.reload(nk)

<module 'netket' from '../netket/__init__.py'>

In [28]:
length = [4, 4]
hex_ = nk.machine.new_hex(np.array(length))

kernel = 1
# sweep_size = 200
decay_factor = 1  # or 'sigmoid decay'



g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)



op, ad2o_o, op_num, label_num  = f.dimer_flip1(length = np.array(length), return_info = True)
op2= f.dimer_flip_old(length = np.array(length))

ad2_bool = np.zeros([ad2o_o.shape[0], ad2o_o.shape[0]], dtype = np.bool)
for l in range(ad2o_o.shape[0]):
    label = ad2o_o[l]
    for op_ in label:
        ad2_bool[l,op_] = True




In [41]:
32 * 3 / 2

48.0

In [62]:
np.argwhere(op._operators[0] == -1)

array([[ 1,  7],
       [ 2,  4],
       [13, 11],
       [14,  8]])

array([[12,  8,  1, 29],
       [ 8,  1,  5,  2],
       [25, 29,  1,  5],
       [ 1,  5,  9, 13],
       [ 5,  9, 16, 20],
       [16,  9, 13, 10],
       [ 9, 13, 17, 21],
       [13, 17, 24, 28],
       [24, 17, 21, 18],
       [17, 21, 25, 29],
       [21, 25, 32,  4],
       [32, 25, 29, 26],
       [ 9,  5,  2, 30],
       [ 5,  2,  6,  3],
       [26, 30,  2,  6],
       [ 2,  6, 10, 14],
       [ 6, 10, 13, 17],
       [13, 10, 14, 11],
       [10, 14, 18, 22],
       [14, 18, 21, 25],
       [21, 18, 22, 19],
       [18, 22, 26, 30],
       [22, 26, 29,  1],
       [29, 26, 30, 27],
       [10,  6,  3, 31],
       [ 6,  3,  7,  4],
       [27, 31,  3,  7],
       [ 3,  7, 11, 15],
       [ 7, 11, 14, 18],
       [14, 11, 15, 12],
       [11, 15, 19, 23],
       [15, 19, 22, 26],
       [22, 19, 23, 20],
       [19, 23, 27, 31],
       [23, 27, 30,  2],
       [30, 27, 31, 28],
       [11,  7,  4, 32],
       [ 1,  8,  4,  7],
       [ 8,  4, 32, 28],
       [16, 12,  8,  4],


In [29]:
ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

ma.hex.ad2o_o = ad2o_o.astype(np.int64)
ma.hex.ad2_bool = ad2_bool

ma._ws[:] = np.random.normal(0, 1e-3)
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

2


In [30]:
V = 1
h = 1

ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))


n_jobs = 8
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op2
    , length = length, n_chains=n_chains, sweep_size = sweep_size*4, kernel = 1, n_jobs=n_jobs, transition = 1)

sr = nk.optimizer.SR(ma, diag_shift=5e-3)
opt = nk.optimizer.Sgd(ma, learning_rate=0.05, decay_factor = decay_factor ,N = n_iter)

gs = nk.Vmc(
hamiltonian=ham,
sampler=sa_mul,
optimizer=opt,
n_samples=10**3,
sr=sr,
n_discard=12*12,
)


# gs.run(n_iter=1000)
# ma.save(parentdir + '/save/ma/'+name)

yoyo
number of core : 8
transtion1
transtion1
transtion1
transtion1
transtion1
transtion1
transtion1
transtion1
discard =  144


In [31]:
sweep_size = 1000
sa_mul.reset()
_ = sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.

In [32]:
gs._samples = sa_mul.generate_samples(10000)

In [33]:
gs._samples.shape

(10000, 1, 32)

In [34]:
eloc, gs._loss_stats = gs._get_mc_stats(gs._ham)


     prepare getting local values 0.006664276123046875
     obtained local values 0.0409238338470459


In [35]:
gs._loss_stats

0.0000022+0.0000000j ± 0.0000016 [σ²=0.0000000]

In [9]:
sa_mul.sa_list[0]._kernel